# Factorizations and other fun
Based on work by Andreas Noack

## Outline
 - Factorizations
 - Special matrix structures
 - Generic linear algebra

Before we get started, let's set up a linear system and use `LinearAlgebra` to bring in the factorizations and special matrix structures.

In [1]:
using LinearAlgebra
A = rand(3, 3)
x = fill(1, (3,))
b = A * x

3-element Array{Float64,1}:
 1.0462981007680332
 2.343559148687405
 1.4365030829350265

## Factorizations

#### LU factorizations
In Julia we can perform an LU factorization
```julia
PA = LU
```
where `P` is a permutation matrix, `L` is lower triangular unit diagonal and `U` is upper triangular, using `lufact`.

Julia allows computing the LU factorization and defines a composite factorization type for storing it.

In [2]:
Alu = lu(A)

LU{Float64,Array{Float64,2}}
L factor:
3×3 Array{Float64,2}:
 1.0        0.0       0.0
 0.0537419  1.0       0.0
 0.453577   0.638179  1.0
U factor:
3×3 Array{Float64,2}:
 0.70726  0.95036    0.685939
 0.0      0.868831   0.0515199
 0.0      0.0       -0.21383

In [3]:
typeof(Alu)

LU{Float64,Array{Float64,2}}

The different parts of the factorization can be extracted by accessing their special properties

In [4]:
Alu.P

3×3 Array{Float64,2}:
 0.0  1.0  0.0
 1.0  0.0  0.0
 0.0  0.0  1.0

In [5]:
Alu.L

3×3 Array{Float64,2}:
 1.0        0.0       0.0
 0.0537419  1.0       0.0
 0.453577   0.638179  1.0

In [6]:
Alu.U

3×3 Array{Float64,2}:
 0.70726  0.95036    0.685939
 0.0      0.868831   0.0515199
 0.0      0.0       -0.21383

Julia can dispatch methods on factorization objects.

For example, we can solve the linear system using either the original matrix or the factorization object.

In [7]:
A\b

3-element Array{Float64,1}:
 1.0
 1.0
 0.9999999999999998

In [8]:
Alu\b

3-element Array{Float64,1}:
 1.0
 1.0
 0.9999999999999998

Similarly, we can calculate the determinant of `A` using either `A` or the factorization object

In [9]:
det(A) ≈ det(Alu)

true

#### QR factorizations

In Julia we can perform a QR factorization
```
A=QR
```

where `Q` is unitary/orthogonal and `R` is upper triangular, using `qrfact`.

In [10]:
Aqr = qr(A)

LinearAlgebra.QRCompactWY{Float64,Array{Float64,2}}
Q factor:
3×3 LinearAlgebra.QRCompactWYQ{Float64,Array{Float64,2}}:
 -0.0488842   0.860431  -0.507216
 -0.90961    -0.248119  -0.333238
 -0.412578    0.445078   0.794786
R factor:
3×3 Array{Float64,2}:
 -0.777542  -1.31603   -0.681965
  0.0        0.994352  -0.0362081
  0.0        0.0       -0.169949

Similarly to the LU factorization, the matrices `Q` and `R` can be extracted from the QR factorization object via

In [11]:
Aqr.Q

3×3 LinearAlgebra.QRCompactWYQ{Float64,Array{Float64,2}}:
 -0.0488842   0.860431  -0.507216
 -0.90961    -0.248119  -0.333238
 -0.412578    0.445078   0.794786

In [12]:
Aqr.R

3×3 Array{Float64,2}:
 -0.777542  -1.31603   -0.681965
  0.0        0.994352  -0.0362081
  0.0        0.0       -0.169949

#### Eigendecompositions

The results from eigendecompositions, singular value decompositions, Hessenberg factorizations, and Schur decompositions are all stored in `Factorization` types.

The eigendecomposition can be computed

In [13]:
Asym = A + A'
AsymEig = eigen(Asym)

Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}
values:
3-element Array{Float64,1}:
 -1.1886355359036576
 -0.2395028075390705
  3.66522832749462
vectors:
3×3 Array{Float64,2}:
  0.604346  -0.682043   -0.411805
 -0.60118   -0.0511964  -0.797472
  0.522827   0.729518   -0.440971

The values and the vectors can be extracted from the Eigen type by special indexing

In [14]:
AsymEig.values

3-element Array{Float64,1}:
 -1.1886355359036576
 -0.2395028075390705
  3.66522832749462

In [15]:
AsymEig.vectors

3×3 Array{Float64,2}:
  0.604346  -0.682043   -0.411805
 -0.60118   -0.0511964  -0.797472
  0.522827   0.729518   -0.440971

Once again, when the factorization is stored in a type, we can dispatch on it and write specialized methods that exploit the properties of the factorization, e.g. that $A^{-1}=(V\Lambda V^{-1})^{-1}=V\Lambda^{-1}V^{-1}$.

In [16]:
inv(AsymEig)*Asym

3×3 Array{Float64,2}:
 1.0           4.44089e-15  3.71925e-15
 5.55112e-17   1.0          9.99201e-16
 0.0          -4.44089e-15  1.0

## Special matrix structures
Matrix structure is very important in linear algebra. To see *how* important it is, let's work with a larger linear system

In [17]:
n = 1000
A = randn(n,n);

Julia can often infer special matrix structure

In [18]:
Asym = A + A'
issymmetric(Asym)

true

but sometimes floating point error might get in the way.

In [19]:
Asym_noisy = copy(Asym)
Asym_noisy[1,2] += 5eps()

1.7963380087552714

In [20]:
issymmetric(Asym_noisy)

false

Luckily we can declare structure explicitly with, for example, `Diagonal`, `Triangular`, `Symmetric`, `Hermitian`, `Tridiagonal` and `SymTridiagonal`.

In [21]:
Asym_explicit = Symmetric(Asym_noisy);

Let's compare how long it takes Julia to compute the eigenvalues of `Asym`, `Asym_noisy`, and `Asym_explicit`

In [22]:
@time eigvals(Asym);

  0.175240 seconds (165.64 k allocations: 15.971 MiB)


In [23]:
@time eigvals(Asym_noisy);

  0.542449 seconds (14 allocations: 7.921 MiB, 1.53% gc time)


In [24]:
@time eigvals(Asym_explicit);

  0.060255 seconds (6.40 k allocations: 8.334 MiB)


In this example, using `Symmetric()` on `Asym_noisy` made our calculations about `5x` more efficient :)

#### A big problem
Using the `Tridiagonal` and `SymTridiagonal` types to store tridiagonal matrices makes it possible to work with potentially very large tridiagonal problems. The following problem would not be possible to solve on a laptop if the matrix had to be stored as a (dense) `Matrix` type.

In [25]:
n = 1_000_000;
A = SymTridiagonal(randn(n), randn(n-1));
@time eigmax(A)

  0.758158 seconds (473.38 k allocations: 206.833 MiB, 9.51% gc time)


6.390371140348492

## Generic linear algebra
The usual way of adding support for numerical linear algebra is by wrapping BLAS and LAPACK subroutines. For matrices with elements of `Float32`, `Float64`, `Complex{Float32}` or `Complex{Float64}` this is also what Julia does.

However, Julia also supports generic linear algebra, allowing you to, for example, work with matrices and vectors of rational numbers.

#### Rational numbers
Julia has rational numbers built in. To construct a rational number, use double forward slashes:

In [26]:
1//2

1//2

#### Example: Rational linear system of equations
The following example shows how linear system of equations with rational elements can be solved without promoting to floating point element types. Overflow can easily become a problem when working with rational numbers so we use `BigInt`s.

In [27]:
Arational = Matrix{Rational{BigInt}}(rand(1:10, 3, 3))/10

3×3 Array{Rational{BigInt},2}:
 3//5   1//10  2//5
 3//10  7//10  3//10
 1//1   9//10  9//10

In [28]:
x = fill(1, 3)
b = Arational*x

3-element Array{Rational{BigInt},1}:
 11//10
 13//10
 14//5

In [29]:
Arational\b

3-element Array{Rational{BigInt},1}:
 1//1
 1//1
 1//1

In [30]:
lu(Arational)

LU{Rational{BigInt},Array{Rational{BigInt},2}}
L factor:
3×3 Array{Rational{BigInt},2}:
 1//1     0//1   0//1
 3//5     1//1   0//1
 3//10  -43//44  1//1
U factor:
3×3 Array{Rational{BigInt},2}:
 1//1    9//10    9//10
 0//1  -11//25   -7//50
 0//1    0//1   -47//440

### Exercises

#### 11.1
What are the eigenvalues of matrix A?

```
A =
[
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36
]
```
and assign it a variable `A_eigv`

In [31]:
using LinearAlgebra

In [32]:
A =
[
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36
]

5×5 Array{Int64,2}:
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36

In [33]:
A_eigv = eigvals(A)

5-element Array{Float64,1}:
 -128.49322764802145
  -55.887784553056875
   42.7521672793189
   87.16111477514521
  542.4677301466143

In [34]:
@assert A_eigv ==  [-128.49322764802145, -55.887784553056875, 42.7521672793189, 87.16111477514521, 542.4677301466143]

#### 11.2
Create a `Diagonal` matrix from the eigenvalues of `A`.

In [35]:
A_diag = Diagonal(A_eigv)

5×5 Diagonal{Float64,Array{Float64,1}}:
 -128.493     ⋅        ⋅        ⋅         ⋅ 
     ⋅     -55.8878    ⋅        ⋅         ⋅ 
     ⋅        ⋅      42.7522    ⋅         ⋅ 
     ⋅        ⋅        ⋅      87.1611     ⋅ 
     ⋅        ⋅        ⋅        ⋅      542.468

In [36]:
@assert A_diag == [-128.49322764802145  0.0      0.0      0.0       0.0;
    0.0    -55.887784553056875   0.0      0.0       0.0;
    0.0      0.0     42.7521672793189   0.0       0.0;
    0.0      0.0      0.0     87.16111477514521    0.0;
    0.0 0.0      0.0      0.0     542.4677301466143;]

#### 11.3
Create a `LowerTriangular` matrix from `A` and store it in `A_lowertri`


In [37]:
A_lowertri = LowerTriangular(A)

5×5 LowerTriangular{Int64,Array{Int64,2}}:
 140    ⋅    ⋅    ⋅   ⋅
  97  106    ⋅    ⋅   ⋅
  74   89  152    ⋅   ⋅
 168  131  144   54   ⋅
 131   36   71  142  36

In [38]:
@assert A_lowertri ==  [140    0    0    0   0;
  97  106    0    0   0;
  74   89  152    0   0;
 168  131  144   54   0;
 131   36   71  142  36]